In [ ]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Multiply,Dropout
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
import pickle

In [ ]:
def loadData(id):
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)


    print('X_train:',X_train.shape)
    print('X_test:',X_test.shape)
    print('y_train',y_train.shape) 
    print('y_test',y_test.shape) 

   
    return X_train,y_train,X_test,y_test


In [ ]:
X_train,y_train,X_test,y_test=loadData(1001)
#input_shape = batch_size,time_steps,input_dim

#batch_size = X_train.shape[0]   #automatically taken no need to specify
time_steps = X_train.shape[1]
input_dim = X_train.shape[2]

In [ ]:
# Defined shared layers as global variables
repeator = RepeatVector(time_steps) #500
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [ ]:
#one step attention

def one_step_attention(a,s_prev):
    
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    e = densor(concat)
    alphas = activator(e)
    context = dotor([alphas, a])
    
    return context

In [ ]:
hid_pre_act = 16
hid_post_act = 96
post_activation_lstm_cell = CuDNNLSTM(hid_post_act,return_state=True)
#output_layer = Dense(128,activation='relu')
#onemore = Dense(64,activation='relu')
fin_out = Dense(6,activation='softmax')

In [ ]:
# model

def model(time_steps,input_dim,hid_post_act,hid_pre_act):
       
    X = Input(shape =(time_steps,input_dim))
    s0 = Input(shape=(hid_post_act,),name='s0')
    c0 = Input(shape=(hid_post_act,),name='c0')
    s=s0
    c=c0
    outputs=[]
    
    
    
    a = CuDNNLSTM(hid_pre_act,return_sequences=True)(X)
    a = Dropout(0.4)(a)
    #6 is final output size
    for t in range(1):
        context =one_step_attention(a,s)
        s, _, c = post_activation_lstm_cell(context, initial_state=[s, c])
        #out = output_layer(s)
        #o = onemore(out)
        k=fin_out(s)
        outputs.append(k)
    
        
    model = Model([X, s0, c0], outputs)
    
    return model       

In [ ]:
model = model(time_steps,input_dim,hid_post_act,hid_pre_act)

In [ ]:
model.summary()

In [ ]:
opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
s0 = np.zeros((7360, hid_post_act))
c0 = np.zeros((7360, hid_post_act))
s0_t = np.zeros((82, hid_post_act))
c0_t = np.zeros((82, hid_post_act))

In [ ]:
model.fit([X_train, s0, c0], y_train, epochs=90,validation_data=([X_test,s0_t,c0_t],y_test))


In [ ]:
K.clear_session()